###imports

In [2]:
import torch

In [3]:
from google.colab import drive

In [4]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import Counter
import numpy as np

In [5]:
!pip install -U -q gspread
!pip install -U -q gspread-dataframe

import gspread
from google.colab import auth
from google.auth import default
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00


In [6]:
import random

In [7]:
from torch.utils.data import Subset

###data loading

In [8]:
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
path_to_csv = '1rqoUNad5grevyfBqJYwjhdYivcVpJs79rOdZo1_EYpM' #make this whatever ID the data is in your drive

In [10]:
# Step 2: Authenticate with Google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Step 3: Access Google Sheets in Shared Drive by its file ID
# Replace 'your_file_id_here' with the actual Google Sheets file ID
file_id = path_to_csv
spreadsheet = gc.open_by_key(file_id)

# Step 4: Open the first sheet (you can change this if needed)
worksheet = spreadsheet.get_worksheet(0)

# Step 5: Extract all cell values as a list of lists
data = worksheet.get_all_values()

total_string = ""

# Step 6: Initialize a dictionary to store unique words
word_dict = {}

current_index = 0

# Step 7: Process each cell to extract words
for row in data:
    for cell in row:
        # Split cell content into words (remove punctuation and make lowercase)
        words = cell.lower().split()
        for word in words:
            # Add word to dictionary with a value of 0 if it's new
            if word not in word_dict:
                word_dict[word] = current_index
                current_index += 1
            total_string += word + " "
        total_string += "\n"
    total_string += "EOP\n"



word_dict["\n"] = len(word_dict)
word_dict["EOP"] = len(word_dict)
# Display the resulting dictionary
print(word_dict)

{'last': 0, 'red': 1, 'in': 2, 'the': 3, 'sky': 4, 'a': 5, 'small': 6, "girl's": 7, 'moon': 8, 'face': 9, 'rises': 10, 'over': 11, 'counter': 12, 'christmas': 13, 'services': 14, 'cellular': 15, 'phone': 16, 'rings': 17, 'out': 18, "handel's": 19, 'messiah': 20, 'passover': 21, 'darkness': 22, '-': 23, 'before': 24, 'buds': 25, 'burst': 26, 'open;': 27, "child's": 28, 'eyes': 29, 'death.': 30, 'night': 31, 'of': 32, 'summer': 33, 'bright': 34, 'full': 35, 'hidden': 36, 'by': 37, 'cloud': 38, 'midnight': 39, 'and': 40, 'my': 41, 'neighbour': 42, 'asks': 43, 'to': 44, 'borrow': 45, 'vacum': 46, 'cleaner': 47, 'yellow': 48, 'walnut': 49, 'leaves': 50, 'slowly': 51, 'appear': 52, 'on': 53, 'lawn--': 54, 'early': 55, 'morning': 56, 'light': 57, 'after': 58, 'its': 59, 'first': 60, 'flight': 61, 'young': 62, "gerfalcon's": 63, 'talons': 64, 'tighter': 65, 'glove': 66, 'sultry': 67, 'afternoon': 68, 'only': 69, 'mailbox': 70, 'shadow': 71, 'crosses': 72, 'dirt': 73, 'road': 74, 'long': 75, 'j

In [12]:
print(total_string[0:100])

last red in the sky 
a small girl's moon face rises 
over the counter 
EOP
christmas services 
a cel


In [ ]:
inverse_word_dict = {}
for key in word_dict.keys():
  inverse_word_dict[word_dict[key]] = key

print(inverse_word_dict)

{0: 'last', 1: 'red', 2: 'in', 3: 'the', 4: 'sky', 5: 'a', 6: 'small', 7: "girl's", 8: 'moon', 9: 'face', 10: 'rises', 11: 'over', 12: 'counter', 13: 'christmas', 14: 'services', 15: 'cellular', 16: 'phone', 17: 'rings', 18: 'out', 19: "handel's", 20: 'messiah', 21: 'passover', 22: 'darkness', 23: '-', 24: 'before', 25: 'buds', 26: 'burst', 27: 'open;', 28: "child's", 29: 'eyes', 30: 'death.', 31: 'night', 32: 'of', 33: 'summer', 34: 'bright', 35: 'full', 36: 'hidden', 37: 'by', 38: 'cloud', 39: 'midnight', 40: 'and', 41: 'my', 42: 'neighbour', 43: 'asks', 44: 'to', 45: 'borrow', 46: 'vacum', 47: 'cleaner', 48: 'yellow', 49: 'walnut', 50: 'leaves', 51: 'slowly', 52: 'appear', 53: 'on', 54: 'lawn--', 55: 'early', 56: 'morning', 57: 'light', 58: 'after', 59: 'its', 60: 'first', 61: 'flight', 62: 'young', 63: "gerfalcon's", 64: 'talons', 65: 'tighter', 66: 'glove', 67: 'sultry', 68: 'afternoon', 69: 'only', 70: 'mailbox', 71: 'shadow', 72: 'crosses', 73: 'dirt', 74: 'road', 75: 'long', 76

In [ ]:
all_poems = []
for row in data:
  new_entry = []
  for cell in row:
    for word in cell.lower().split():
      new_entry.append(word)
    new_entry.append("\n")
  new_entry.append("EOP")
  all_poems.append(new_entry)

In [ ]:
def generate_poem_dataset(poems, converter):
    """
    Generates a dataset of input sequences and target labels from a list of poems.

    Args:
        poems (list of lists): Each inner list is a poem represented as a list of word indices.

    Returns:
        data (list of lists): Each inner list is an input sequence of word indices up to a certain point in a poem.
        targets (list): Each item is the next word (target label) following the corresponding input sequence in 'data'.
    """
    data = []
    targets = []

    # Loop through each poem in the dataset
    for poem in poems:
        # Generate all possible input-target pairs within each poem
        for i in range(1, len(poem)):
            input_sequence = poem[:i]  # All words up to the i-th word (not inclusive)
            target_word = poem[i]      # The next word in the sequence

            for index, element in enumerate(input_sequence):
              input_sequence[index] = converter[element]

            target_word = converter[target_word]

            # Append to dataset lists
            data.append(input_sequence)
            targets.append(target_word)

    return data, targets


class PoemDataset(Dataset):
    def __init__(self, data, targets):
        """
        Args:
            data (list of lists): Each inner list is an input sequence of word indices.
            targets (list): Each item is the next word (target label) following the corresponding input sequence.
        """
        self.data = [torch.tensor(seq, dtype=torch.long) for seq in data]
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]


data, targets = generate_poem_dataset(all_poems, word_dict)

In [ ]:
# for i in range(30):
#   print(data[i])
#   true_sequence = ""
#   for val in data[i]:
#     true_sequence += inverse_word_dict[val] + " "
#   print(true_sequence)
#   print('---------------')
#   print(targets[i])
#   print(inverse_word_dict[targets[i]])
#   print("\n\n")

###define model

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """
    Pads sequences to the length of the longest sequence in the batch.

    Args:
        batch (List[Tuple[Tensor, Tensor]]): Each tuple contains (input_sequence, target_word).

    Returns:
        padded_sequences (Tensor): Padded input sequences of shape (batch_size, max_seq_length).
        lengths (Tensor): Original lengths of each sequence before padding.
        targets (Tensor): Target words of shape (batch_size).
    """
    sequences, targets = zip(*batch)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=0)  # Assuming 0 is the padding index
    targets = torch.stack(targets)
    return padded_sequences, lengths, targets


In [ ]:
full_dataset = PoemDataset(data, targets)
train_dataset = full_dataset
test_dataset = Subset(full_dataset, range(80000, 90000))

BATCH_SIZE = 64

# Initialize the DataLoader with the custom collate_fn
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,          # Adjust based on your system
    collate_fn=collate_fn,  # Use the custom collate function
    pin_memory=True         # If using GPU
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,          # Adjust based on your system
    collate_fn=collate_fn,  # Use the custom collate function
    pin_memory=True         # If using GPU
)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# ----------------------------
# Define the Model
# ----------------------------
class LSTMClassifierPacked(nn.Module):
    def __init__(self, num_classes, embedding_dim, hidden_dim, num_layers, padding_idx=0, dropout=0.5):
        super(LSTMClassifierPacked, self).__init__()
        self.embedding = nn.Embedding(num_classes, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0  # Dropout only between LSTM layers
        )
        self.fc = nn.Linear(hidden_dim, num_classes)
        # No Softmax layer since CrossEntropyLoss applies it internally

    def forward(self, x, lengths):
        """
        Args:
            x (Tensor): Padded input sequences of shape (batch_size, max_seq_length).
            lengths (Tensor): Original lengths of each sequence before padding.

        Returns:
            out (Tensor): Logits for each class of shape (batch_size, num_classes).
        """
        # x: [batch_size, max_seq_length]
        embeds = self.embedding(x)  # [batch_size, max_seq_length, embedding_dim]

        # Pack the padded sequence
        packed_input = nn.utils.rnn.pack_padded_sequence(embeds, lengths.cpu(), batch_first=True, enforce_sorted=False)

        # LSTM
        packed_output, (hidden, cell) = self.lstm(packed_input)

        # hidden: [num_layers, batch_size, hidden_dim]
        # Take the last layer's hidden state
        last_hidden = hidden[-1]  # [batch_size, hidden_dim]

        out = self.fc(last_hidden)  # [batch_size, num_classes]
        return out

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn,
    pin_memory=True  # If using GPU
)
# ----------------------------
# Initialize the Model, Loss Function, and Optimizer
# ----------------------------
# Define hyperparameters
NUM_CLASSES = 28082        # Adjust as per your requirement
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_LAYERS = 2
NUM_EPOCHS = 30
LEARNING_RATE = 0.001
PADDING_IDX = 0            # Ensure this matches the padding_value in collate_fn

# Initialize the model, loss function, and optimizer
model = LSTMClassifierPacked(
    num_classes=NUM_CLASSES,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    padding_idx=PADDING_IDX,
    dropout=0.5
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Check if CUDA is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

# ----------------------------
# Training Loop
# ----------------------------
max_accuracy = 0
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, lengths, targets) in enumerate(train_loader):
        inputs = inputs.to(device)       # [batch_size, max_seq_length]
        lengths = lengths.to(device)     # [batch_size]
        targets = targets.to(device)     # [batch_size]

        # Forward pass
        outputs = model(inputs, lengths)  # [batch_size, num_classes]
        # print(f"targets are {targets}")
        # print(f"outputs are {outputs}")
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()

        # Optional: Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

        if (batch_idx + 1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%")

            if float(100 * correct / total) > float(max_accuracy):
              max_accuracy = 100 * correct / total
              print(f"New max accuracy: {max_accuracy}%")
              torch.save(model.state_dict(), "model_parameters.pth")
            # Reset counters
            correct = 0
            total = 0

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] completed. Average Loss: {avg_loss:.4f}")

print("Training Finished!")

torch.save(model.state_dict(), "model_parameters.pth")


Using device: cuda
Epoch [1/30], Step [100/1250], Loss: 7.0968, Accuracy: 18.66%
New max accuracy: 18.65625%
Epoch [1/30], Step [200/1250], Loss: 6.3903, Accuracy: 19.28%
New max accuracy: 19.28125%
Epoch [1/30], Step [300/1250], Loss: 5.7728, Accuracy: 26.02%
New max accuracy: 26.015625%
Epoch [1/30], Step [400/1250], Loss: 5.5914, Accuracy: 25.81%
Epoch [1/30], Step [500/1250], Loss: 6.5098, Accuracy: 26.05%
New max accuracy: 26.046875%
Epoch [1/30], Step [600/1250], Loss: 5.5799, Accuracy: 26.17%
New max accuracy: 26.171875%
Epoch [1/30], Step [700/1250], Loss: 5.4436, Accuracy: 27.12%
New max accuracy: 27.125%
Epoch [1/30], Step [800/1250], Loss: 5.3559, Accuracy: 26.97%
Epoch [1/30], Step [900/1250], Loss: 5.6033, Accuracy: 25.97%
Epoch [1/30], Step [1000/1250], Loss: 6.4249, Accuracy: 26.84%
Epoch [1/30], Step [1100/1250], Loss: 6.2623, Accuracy: 27.25%
New max accuracy: 27.25%
Epoch [1/30], Step [1200/1250], Loss: 4.8000, Accuracy: 25.81%
Epoch [1/30] completed. Average Loss: 5.

In [ ]:
def convert_sentence_to_nums(sentence):
  nums = []
  for element in sentence.split():
    nums.append(word_dict[element])
  return nums

def convert_nums_to_sentence(nums):
  sentence = ""
  for num in nums:
    sentence += inverse_word_dict[num] + " "
  return sentence

In [ ]:
start = "evan has a stick"
start_nums = convert_sentence_to_nums(start)

for i in range(1000):
  next_word = model(torch.tensor(start_nums).unsqueeze(0).to(device), torch.tensor([len(start_nums)]).to(device))
  _, next_word = torch.topk(next_word.data, 5)
  #print(next_word[0][1])
  new_word_num = next_word[0][0].item()
  start_nums.append(new_word_num)
  #print(convert_nums_to_sentence(start_nums))
  if new_word_num == word_dict["EOP"]:
    break

print(convert_nums_to_sentence(start_nums))

evan has a stick 
 of the sea of white life suit 
 in the high goes 
 EOP 


In [21]:
import random
from collections import defaultdict, Counter
import re

def build_markov_chain(text):
    # Split the text into words
    words = re.findall(r"\w+\'\w+|\w+|\n", text)

    # Create a dictionary to hold the Markov chain with frequencies
    markov_chain = defaultdict(Counter)

    # Populate the Markov chain with counts of each following word
    for i in range(len(words) - 1):
        current_word = words[i]
        next_word = words[i + 1]
        markov_chain[current_word][next_word] += 1

    return markov_chain

def generate_text(markov_chain, start_word="START"):
    # Initialize the generated text and the first word
    generated_words = []
    current_word = start_word

    # Generate words until "EOP" is found
    while current_word != "EOP":
        generated_words.append(current_word)
        next_words = markov_chain.get(current_word, None)

        # If there are no next words, stop generation
        if not next_words:
            break

        # Choose the next word based on frequency distribution
        current_word = random.choices(
            population=list(next_words.keys()),
            weights=list(next_words.values())
        )[0]

    # Convert the list of words to a single string
    generated_text = ' '.join(generated_words)
    return generated_text

# Example usage
input_text = total_string
markov_chain = build_markov_chain(input_text)
for i in range(10):
  generated_string = generate_text(markov_chain, start_word="the")
  print(generated_string)
  print("\n---------------------------------------------\n")


the 
 play you have a night of dumbo 
 more christians 
 the world 
 sudden 
 garbage front of how many of you 
 goes 


---------------------------------------------

the process 


---------------------------------------------

the other way done for playing with me a double tap and i don't know what people find the chopping off a list great thursday thankyou 
 of pride in 
 lebron is something about 
 get 


---------------------------------------------

the solo artist 


---------------------------------------------

the wheel of nun on me home page is getting nba sorry i make you remember quite often 
 sometimes you your dreams 
 ok i can't leave you a ear 


---------------------------------------------

the shirt 
 august 
 and meet 
 with me mad love this been looking forward 


---------------------------------------------

the 
 your dad about 
 on 


---------------------------------------------

the world i wanna do come with pointy 
 really think 


----------------------

In [45]:
import random
from collections import defaultdict, Counter
import re

def build_markov_chain(text, depth=3):
    # Split the text into words and preserve newline characters
    words = re.findall(r"\w+\'\w+|\w+|\n", text)

    # Create a dictionary to hold the Markov chain with frequencies
    markov_chain = defaultdict(Counter)

    # Populate the Markov chain with n-grams of specified depth
    for i in range(len(words) - depth):
        for d in range(1, depth + 1):
            current_state = tuple(words[i:i + d])  # Tuple of n-grams as the state
            next_word = words[i + d]
            markov_chain[current_state][next_word] += 1

    return markov_chain

def generate_text(markov_chain, start_words, depth=3, max_words=50):
    # Initialize the generated text with the starting words
    generated_words = start_words.split()

    current_state = tuple(start_words)

    # Generate words until "EOP" is found or we reach max_words
    for _ in range(max_words):
        # Try to find a next word using the full depth, then lower depths if needed
        next_word = None
        for d in range(depth, 0, -1):
            current_state = tuple(generated_words[-d:])
            next_words = markov_chain.get(current_state, None)
            if next_words:
                # Choose the next word based on frequency distribution
                next_word = random.choices(
                    population=list(next_words.keys()),
                    weights=list(next_words.values())
                )[0]
                break

        # If no next word is found, stop generation
        if not next_word:
            break

        # Add the chosen word to the generated text
        generated_words.append(next_word)

        # Stop if the word "EOP" is generated
        if next_word == "EOP":
            break

    # Join words to create the final text
    generated_text = ' '.join(generated_words)
    return generated_text

# Example usage
input_text = total_string
depth = 4

markov_chain = build_markov_chain(input_text, depth=depth)




In [47]:
# Set the starting words for the poem (it should be a tuple of the first few words matching the depth)
start_words = "so"  # Change this to whatever sequence you'd like to start with

for i in range(10):
  generated_poem = generate_text(markov_chain, start_words=start_words, depth=depth)
  print("Generated poem:", generated_poem)
  print("\n---------------------------------------------\n")

Generated poem: so brightly in her eyes that she felt she might go blind Her body spasmed desperately sweat popping up on her skin from the sheer strain of her writhing muscles 
 
 Just when she thought she could take it no more and the little bursts of pain would overwhelm everything

---------------------------------------------

Generated poem: so brightly in her eyes that she felt she might go blind Her body spasmed desperately sweat popping up on her skin from the sheer strain of her writhing muscles 
 
 Just when she thought she might break Mr Charles pulled his mouth from her body a wet grin gleaming

---------------------------------------------

Generated poem: so mean he said sulkily 
 
 Mr Charles lips found her clit right as Liam s fingers found her nipple Mr Charles flicked that tight bundle of nerves with this tongue Liam flicked the already overstimulated bud with his fingers And between then Josephine whimpered and panted straining and struggling

----------------------

In [1]:
import re

text = "hello I'm potato\nI like pie"
tokens = re.findall(r"\w+\'\w+|\w+|\n", text)
print(tokens)

['hello', "I'm", 'potato', '\n', 'I', 'like', 'pie']
